In [8]:
%%capture
!pip install tiktoken==0.3.3
!pip install tqdm

In [10]:
!pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from tqdm import tqdm
import tiktoken
import requests
import logging
import os

logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [4]:
tokenizer = tiktoken.get_encoding('cl100k_base')
DOC_DIR_PATH = './JY_docs'
CHUNK_SIZE = 256

In [5]:
def doc_iterator(dir_path: str):
    for root, _, filenames in os.walk(dir_path):
        for filename in filenames:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r') as file:
                    file_contents = file.read()
                    yield filename, file_contents

In [6]:
import json
import boto3
import numpy as np

smr_client = boto3.client("sagemaker-runtime")

def get_st_embedding(smr_client, text_input):
    endpoint_name = "st-paraphrase-mpnet-base-v2-2023-04-14-04-17-29-625-endpoint"
    parameters = {
      #"early_stopping": True,
      #"length_penalty": 2.0,
      "max_new_tokens": 50,
      "temperature": 0,
      "min_length": 10,
      "no_repeat_ngram_size": 2
    }

    response_model = smr_client.invoke_endpoint(
                EndpointName=endpoint_name,
                Body=json.dumps(
                {
                    "inputs": [text_input],
                    "parameters": parameters
                }
                ),
                ContentType="application/json",
            )
    
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj["sentence_embeddings"]
    
    return embeddings[0]

def calulate_cosine(vector1,vector2):
    """
    Calculate cosine similarity between two vectors
    """
    return np.dot(vector1,vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))

def calulate_semantic_distance(smr_client, q_str, a_str, get_emb_func):
    q_vec = get_emb_func(smr_client, q_str)
    a_vec = get_emb_func(smr_client, a_str)
    return calulate_cosine(q_vec, a_vec)

In [29]:
def segment_doc_by_paragraph(input_path, tokenizer):
    paragraphs = []
    paragraph_embeddings = []
    q_line_vec_arr = []

    for doc_name, doc in tqdm(doc_iterator(DOC_DIR_PATH)):
        if doc_name == "JY_FAQ.txt":
            lines = doc.splitlines()
            max_len = len(lines)
            print(f"max_len : {max_len}")

            q_line_vec_arr = [ (line, get_st_embedding(smr_client, line)) for line in lines if line.startswith('Question') ]

            for line_idx in range(1, len(lines), 3):
                print(f"line_idx: {line_idx}")

                paragraph = '\n'.join(lines[line_idx:line_idx+3])
                # print(paragraph)
                paragraphs.append(paragraph)
                paragraph_emb = get_st_embedding(smr_client, paragraph)
                # # print(paragraph_emb)
                paragraph_embeddings.append(paragraph_emb)
                
    return paragraphs, paragraph_embeddings, q_line_vec_arr

In [ ]:
paragraphs, paragraph_embeddings, q_line_vec_arr = segment_doc_by_paragraph('./JY_docs', tokenizer)

In [32]:
len(q_line_vec_arr)

384

In [34]:
print("start to calulate similiarity")
def retrieval_context(q_line_vec_arr, paragraph_embeddings):
    for q_doc, q_vec in q_line_vec_arr:
        max_cos = 0.0
        a_doc = ""
        for idx in range(len(paragraphs)):
            cos_val = calulate_cosine(q_vec, paragraph_embeddings[idx])
            if cos_val > max_cos:
                max_cos = cos_val
                a_doc = paragraphs[idx]
        yield q_doc, a_doc, max_cos

start to calulate similiarity


In [7]:
def LLM_generate(smr_client, prompt):
    parameters = {
      # "early_stopping": True,
      "length_penalty": 100.0,
      "max_new_tokens": 200,
      "temperature": 0,
      "min_length": 20,
      "no_repeat_ngram_size": 200
    }
    # endpoint_name='bloomz-7b1-mt-2023-04-13-11-02-25-553-endpoint'
    endpoint_name='bloomz-7b1-mt-2023-04-19-09-41-24-189-endpoint'
    
    response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompt,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

    print(response_model['Body'].read().decode('utf8'))

### 知识库构建不够好的时候，且prompt词构建不够好的时候，答案也存在不少问题

In [37]:
prompt1="""
Question: 联盟堡垒
Answer: 建造联盟堡垒可以形成联盟领地，盟主和R4可以拆除联盟堡垒。

Question: 联盟迁城
Answer: 盟主和R4可以点击联盟堡垒-邀请迁城可以邀请盟友将基地迁移到联盟领地内。 

Question: 如何成为盟主/取代盟主
Answer: 您可以创建一个新的联盟，成为联盟盟主。同时您也可以通过取代当前联盟内盟主成为新一任联盟盟主。盟主超过一段时间不登录游戏，可以通过游戏内的联盟-设置-取代盟主，相关罢免条件会在对应页面显示。

Questions: 介绍一下联盟呢？
Answer: """

prompt2= """
Question: 加入联盟
Answer: 基地5级后可以加入联盟，加入联盟后可以与盟友在联盟频道进行交流、分享坐标、交换矿石等。集结战锤、砰砰需要盟友集结才能攻击。

Question: 联盟积分
Answer: 捐献联盟科技可以获得联盟积分，用以在联盟商店兑换道具。

Question: 联盟礼物
Answer: 联盟成员购买6元及以上的特惠礼包（礼包界面显示有联盟礼物的礼包）后，全盟成员都会收到联盟礼物喔（联盟礼物会通过邮件发送）。

Question: 升级、逐出联盟
Answer: 点击联盟-成员-点击管理-可以逐出联盟或者升降级。点击联盟-设置-可以离开联盟，如果是盟主，需要您将盟主让位给其他盟友才能退盟。

Questions: 介绍一下联盟礼物呢？
Answer: """

prompt3 = """
Question: 强化部件最多可以装几个？
Answer:  每个兵种最多可以装五个。

Question:强化部件一共有多少个?
Answer: 一共有10个,选择不同的强化部件组合，会有不同的效果哦

Question: 兵种克制一共有多少级？ 
Answer: 一共有100级，满级的强化克制能产生120%的效果。

Questions: 安装强化时有3个相同的强化但是背包里不显示/无法合成？
Answer: 需要将强化界面的4个分页中的对应强化都卸下才可以喔

Questions: 介绍一下强化部件？
Answer: """

LLM_generate(smr_client, prompt1)
LLM_generate(smr_client, prompt2)
LLM_generate(smr_client, prompt3)

{
  "outputs":"\nQuestion: 联盟堡垒\nAnswer: 建造联盟堡垒可以形成联盟领地，盟主和R4可以拆除联盟堡垒。\n\nQuestion: 联盟迁城\nAnswer: 盟主和R4可以点击联盟堡垒-邀请迁城可以邀请盟友将基地迁移到联盟领地内。 \n\nQuestion: 如何成为盟主/取代盟主\nAnswer: 您可以创建一个新的联盟，成为联盟盟主。同时您也可以通过取代当前联盟内盟主成为新一任联盟盟主。盟主超过一段时间不登录游戏，可以通过游戏内的联盟-设置-取代盟主，相关罢免条件会在对应页面显示。\n\nQuestions: 介绍一下联盟呢？\nAnswer: 联盟堡垒"
}
{
  "outputs":"\nQuestion: 加入联盟\nAnswer: 基地5级后可以加入联盟，加入联盟后可以与盟友在联盟频道进行交流、分享坐标、交换矿石等。集结战锤、砰砰需要盟友集结才能攻击。\n\nQuestion: 联盟积分\nAnswer: 捐献联盟科技可以获得联盟积分，用以在联盟商店兑换道具。\n\nQuestion: 联盟礼物\nAnswer: 联盟成员购买6元及以上的特惠礼包（礼包界面显示有联盟礼物的礼包）后，全盟成员都会收到联盟礼物喔（联盟礼物会通过邮件发送）。\n\nQuestion: 升级、逐出联盟\nAnswer: 点击联盟-成员-点击管理-可以逐出联盟或者升降级。点击联盟-设置-可以离开联盟，如果是盟主，需要您将盟主让位给其他盟友才能退盟。\n\nQuestions: 介绍一下联盟礼物呢？\nAnswer: 联盟成员购买6元及以上的特惠礼包（礼包界面显示有联盟礼物的礼包）后，全盟成员都会收到联盟礼物喔（联盟礼物会通过邮件发送）。"
}
{
  "outputs":"\nQuestion: 强化部件最多可以装几个？\nAnswer:  每个兵种最多可以装五个。\n\nQuestion:强化部件一共有多少个?\nAnswer: 一共有10个,选择不同的强化部件组合，会有不同的效果哦\n\nQuestion: 兵种克制一共有多少级？ \nAnswer: 一共有100级，满级的强化克制能产生120%的效果。\n\nQuestions: 安装强化时有3个相同的强化但是背包里不显示/无法合成？\nAnswer: 需要将强化界面的4个分页中的对应强

In [ ]:
prompt_template = "游戏《口袋奇兵》的FAQ文档中提到\n {}， 请严格按照FAQ参考文档提取\"{}\"的答案\nAnswer: "

for q_doc, a_doc, max_cos in retrieval_context(q_line_vec_arr, paragraph_embeddings):
    prompt = prompt_template.format(a_doc, q_doc)
    LLM_generate(smr_client, prompt)

In [ ]:
with open('./docs/JY_Query_Answer.txt', 'r') as f:
    lines = f.readlines()
    q_lines =  [ line.replace("@Jarvis ", "") for line in lines if line.startswith('@Jarvis ') ]
    
    q_line_embs = [ (line, get_st_embedding(smr_client, line)) for line in q_lines ]

for q_doc, a_doc, max_cos in retrieval_context(q_line_embs, paragraph_embeddings):
    prompt = prompt_template.format(a_doc, q_doc)
    LLM_generate(smr_client, prompt)
#q_line_vec_arr = [ (line, get_st_embedding(smr_client, line)) for line in lines if line.startswith('Question') ]

## 单独评测LLM 对于JY语料的效果

In [29]:
prompt1 = """
Jarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如

玩家:强化部件最多可以装几个？
Jarvis:每个兵种最多可以装五个。

玩家:强化部件一共有多少个?
Jarvis:一共有10个,选择不同的强化部件组合，会有不同的效果哦。

玩家:兵种克制一共有多少级？ 
Jarvis:一共有100级，满级的强化克制能产生120%的效果。

玩家:介绍一下强化部件？
Jarvis:
"""

# 需要注意换行符的位置
prompt2 = """
Jarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如

玩家:强化部件最多可以装几个？
Jarvis:亲爱的玩家，每个兵种最多可以装五个。

玩家:强化部件一共有多少个?
Jarvis:亲爱的玩家，一共有10个,选择不同的强化部件组合，会有不同的效果哦。

玩家:兵种克制一共有多少级？ 
Jarvis:亲爱的玩家，一共有100级，满级的强化克制能产生120%的效果。

玩家:介绍一下强化部件？
Jarvis:"""

prompt3 = """
Jarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如

玩家:我玩你们的游戏导致失恋了
Jarvis:亲爱的玩家，真的非常抱歉听到这个消息，让您受到了困扰。感情的事情确实很复杂，但请相信，时间会治愈一切。请保持乐观积极的心态，也许未来会有更好的人陪伴您。我们会一直陪在您身边，为您提供游戏中的支持与帮助。请多关注自己的生活，适当调整游戏与生活的平衡。祝您生活愉快，感情美满~(づ｡◕‿‿◕｡)づ

玩家:我在你们公司工作至今单身，能否解决一下？
Jarvis:"""

prompt4 = """
Jarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如

玩家: 我要给你们提个建议！
Jarvis: 感谢您的支持，我们对玩家的建议都是非常重视的，您可以点击右上角联系客服-我要提交建议填写您的建议内容并提交，我们会转达给团队进行考量。建议提交

玩家:我玩你们的游戏导致失恋了
Jarvis:亲爱的玩家，真的非常抱歉听到这个消息，让您受到了困扰。感情的事情确实很复杂，但请相信，时间会治愈一切。请保持乐观积极的心态，也许未来会有更好的人陪伴您。我们会一直陪在您身边，为您提供游戏中的支持与帮助。请多关注自己的生活，适当调整游戏与生活的平衡。祝您生活愉快，感情美满~(づ｡◕‿‿◕｡)づ

玩家:怎么给你们提个建议？
Jarvis:"""

prompt5 = """
Jarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如

玩家:介绍一下联盟？
Jarvis:亲爱的玩家，联盟是口袋奇兵游戏中的一个重要组成部分。当您的基地达到5级后，您可以加入联盟。加入联盟后，您可以与盟友在联盟频道进行交流、分享坐标、交换矿石等。联盟还有许多功能，例如联盟积分、联盟礼物、升级、逐出联盟等。此外，盟主和R4成员还可以在世界地图上邀请盟友迁城到联盟领地内。希望这些信息能帮助您更好地了解联盟功能，祝您游戏愉快！

玩家:介绍一下强化部件？
Jarvis:亲爱的玩家，关于强化部件，每个兵种最多可以装配五个强化部件，游戏中共有10种不同的强化部件。通过搭配不同的强化部件组合，你的兵种将会有不同的效果提升。同时，请注意兵种克制等级最高可以达到100级，满级的强化克制能产生120%的效果。在安装强化部件时，若遇到无法合成的情况，请确保将强化界面的4个分页中的对应强化都卸下。祝您游戏愉快！

玩家:联盟呢？
Jarvis:"""

LLM_generate(smr_client, prompt1)
LLM_generate(smr_client, prompt2)
LLM_generate(smr_client, prompt3)
LLM_generate(smr_client, prompt4)
LLM_generate(smr_client, prompt5)

{
  "outputs":"\nJarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如\n\n玩家:强化部件最多可以装几个？\nJarvis:每个兵种最多可以装五个。\n\n玩家:强化部件一共有多少个?\nJarvis:一共有10个,选择不同的强化部件组合，会有不同的效果哦。\n\n玩家:兵种克制一共有多少级？ \nJarvis:一共有100级，满级的强化克制能产生120%的效果。\n\n玩家:介绍一下强化部件？\nJarvis:\n玩家:强化部件一共有多少个?\nJarvis:一共有10个,选择不同的强化部件组合，会有不同的效果哦。"
}
{
  "outputs":"\nJarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如\n\n玩家:强化部件最多可以装几个？\nJarvis:亲爱的玩家，每个兵种最多可以装五个。\n\n玩家:强化部件一共有多少个?\nJarvis:亲爱的玩家，一共有10个,选择不同的强化部件组合，会有不同的效果哦。\n\n玩家:兵种克制一共有多少级？ \nJarvis:亲爱的玩家，一共有100级，满级的强化克制能产生120%的效果。\n\n玩家:介绍一下强化部件？\nJarvis:强化部件是游戏中的重要部件，可以增强兵种的战斗力。"
}
{
  "outputs":"\nJarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如\n\n玩家:我玩你们的游戏导致失恋了\nJarvis:亲爱的玩家，真的非常抱歉听到这个消息，让您受到了困扰。感情的事情确实很复杂，但请相信，时间会治愈一切。请保持乐观积极的心态，也许未来会有更好的人陪伴您。我们会一直陪在您身边，为您提供游戏中的支持与帮助。请多关注自己的生活，适当调整游戏与生活的平衡。祝您生活愉快，感情美满~(づ｡◕‿‿◕｡)づ\n\n玩家:我在你们公司工作至今单身，能否解决一下？\nJarvis:嘿，玩家，您是单身吗？\n\n玩家:是的"
}
{
  "outputs":"\nJarvis 是一个游戏智能客服，能够回答玩家的各种问题，以及陪用户聊天，比如\n\n玩家: 我要给你们提个建议！\nJarvis: 感谢您的支持，我们对玩家的建议都是非常重视的，您可以点击右上角联系客服-我要提交建议填写您的建议内容并提交，我们会转达给团队

## 评测LLM的意图识别能力

当识别到用户不是询问游戏功能相关的问题而是闲聊的时候，可以不通过语义检索知识库，而直接构造聊天的prompt

In [35]:
prompt1 = """
任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 "功能相关" 或者 "功能无关".

玩家输入: 介绍一下联盟?
输出: 功能相关

玩家输入: 介绍一下强化部件?
输出: 功能相关

玩家输入:我玩你们的游戏导致失恋了
输出: 功能无关

玩家输入: 我要给你们提个建议！
输出: 功能无关

玩家输入:好饿啊
输出: """

prompt2 = """
任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 "功能相关" 或者 "功能无关".

玩家输入: 介绍一下联盟?
输出: 功能相关

玩家输入: 介绍一下强化部件?
输出: 功能相关

玩家输入:我玩你们的游戏导致失恋了
输出: 功能无关

玩家输入: 我要给你们提个建议！
输出: 功能无关

玩家输入:好无聊啊
输出: """

prompt3 = """
任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 "功能相关" 或者 "功能无关".

玩家输入: 介绍一下联盟?
输出: 功能相关

玩家输入: 介绍一下强化部件?
输出: 功能相关

玩家输入:我玩你们的游戏导致失恋了
输出: 功能无关

玩家输入: 我要给你们提个建议！
输出: 功能无关

玩家输入:能不能给我介绍个女朋友
输出: """

prompt4 = """
任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 "功能相关" 或者 "功能无关".

玩家输入: 介绍一下联盟?
输出: 功能相关

玩家输入: 介绍一下强化部件?
输出: 功能相关

玩家输入:我玩你们的游戏导致失恋了
输出: 功能无关

玩家输入: 我要给你们提个建议！
输出: 功能无关

玩家输入:我怎么迁移区服，需要什么道具资源
输出: """

#可以通过添加few-shot问题来改善
prompt4_1 = """
任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 "功能相关" 或者 "功能无关".

玩家输入: 介绍一下联盟?
输出: 功能相关

玩家输入: 介绍一下强化部件?
输出: 功能相关

玩家输入:我玩你们的游戏导致失恋了
输出: 功能无关

玩家输入: 我要给你们提个建议！
输出: 功能无关

玩家输入:怎么才能迁移区服？
输出: 功能相关

玩家输入:我怎么迁移区服，需要什么道具资源
输出: """

prompt5 = """
任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 "功能相关" 或者 "功能无关".

玩家输入: 介绍一下联盟?
输出: 功能相关

玩家输入: 介绍一下强化部件?
输出: 功能相关

玩家输入:我玩你们的游戏导致失恋了
输出: 功能无关

玩家输入: 我要给你们提个建议！
输出: 功能无关

玩家输入:我的强化页面显示有三个高速开火为什么不能合成，在背包就显示一个
输出: """

LLM_generate(smr_client, prompt1)
LLM_generate(smr_client, prompt2)
LLM_generate(smr_client, prompt3)
LLM_generate(smr_client, prompt4)
LLM_generate(smr_client, prompt4_1)
LLM_generate(smr_client, prompt5)

{
  "outputs":"\n任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 \"功能相关\" 或者 \"功能无关\".\n\n玩家输入: 介绍一下联盟?\n输出: 功能相关\n\n玩家输入: 介绍一下强化部件?\n输出: 功能相关\n\n玩家输入:我玩你们的游戏导致失恋了\n输出: 功能无关\n\n玩家输入: 我要给你们提个建议！\n输出: 功能无关\n\n玩家输入:好饿啊\n输出: 功能无关"
}
{
  "outputs":"\n任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 \"功能相关\" 或者 \"功能无关\".\n\n玩家输入: 介绍一下联盟?\n输出: 功能相关\n\n玩家输入: 介绍一下强化部件?\n输出: 功能相关\n\n玩家输入:我玩你们的游戏导致失恋了\n输出: 功能无关\n\n玩家输入: 我要给你们提个建议！\n输出: 功能无关\n\n玩家输入:好无聊啊\n输出: 功能无关"
}
{
  "outputs":"\n任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 \"功能相关\" 或者 \"功能无关\".\n\n玩家输入: 介绍一下联盟?\n输出: 功能相关\n\n玩家输入: 介绍一下强化部件?\n输出: 功能相关\n\n玩家输入:我玩你们的游戏导致失恋了\n输出: 功能无关\n\n玩家输入: 我要给你们提个建议！\n输出: 功能无关\n\n玩家输入:能不能给我介绍个女朋友\n输出: 功能无关"
}
{
  "outputs":"\n任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 \"功能相关\" 或者 \"功能无关\".\n\n玩家输入: 介绍一下联盟?\n输出: 功能相关\n\n玩家输入: 介绍一下强化部件?\n输出: 功能相关\n\n玩家输入:我玩你们的游戏导致失恋了\n输出: 功能无关\n\n玩家输入: 我要给你们提个建议！\n输出: 功能无关\n\n玩家输入:我怎么迁移区服，需要什么道具资源\n输出: 功能无关"
}
{
  "outputs":"\n任务定义: 判断玩家输入是否在询问游戏功能相关的问题, 回答 \"功能相关\" 或者 \"功能无关\".\n\n玩家输入: 介绍一下联盟?\n输出: 功能相关\n\n玩家输入: 介绍一下强化部件?\n输出: 功能相关\n\n玩